In [1]:
from __future__ import print_function

import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.constraints import nonneg
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from sklearn.utils.extmath import softmax
import tensorflow as tf
from numba import jit
import time
import math

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 20

In [3]:
# train the model
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
# model.add(Dense(256, use_bias=0, activation='relu', input_shape=(784,)))
# model.add(Dense(256, use_bias=0, activation='relu'))
# model.add(Dense(256, use_bias=0, activation='relu'))

model.add(Dense(256, use_bias=0, activation='relu', input_shape=(784,)))
model.add(Dense(256, use_bias=0, activation='relu'))
model.add(Dense(256, use_bias=0, activation='relu'))

model.add(Dense(num_classes, use_bias=0, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               200704    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65536     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2560      
Total params: 334,336
Trainable params: 334,336
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 4s 71us/step - loss: 0.2548 - acc: 0.9221 - val_loss: 0.1235 - val_acc: 0.9616
Epoch 2/20
60000/60000 [==============================] - 2s 29us/step - loss: 0.0969 - acc: 0.9704 - val_loss: 0.1066 - val_acc: 0.9652
Epoch 3/20
60000/60000 [==============================] - 2s 30us/step - loss: 0.0640 - acc: 0.9798 - val_loss: 0.0711 - val_acc: 0.9798
Epoch 4/20
60000/60000 [==============================] - 2s 31us/step - loss: 0.0468 - acc: 0.9858 - val_loss: 0.0921 - val_acc: 0.9772
Epoch 5/20
60000/60000 [==============================] - 2s 30us/step - loss: 0.0357 - acc: 0.9888 - val_loss: 0.0883 - val_acc: 0.9762
Epoch 6/20
60000/60000 [==============================] - 2s 29us/step - loss: 0.0301 - acc: 0.9908 - val_loss: 0.0889 - val_acc: 0.9785
Epoch 7/20
60000/60000 [==============================] - 2s 30us/step - loss: 0.0251 - acc: 0.9926 - val_loss: 0.0951 - val_acc

In [16]:
model.save_weights('my_model_weights.h5')

In [17]:
model2 = Sequential()
model2.add(Dense(256, use_bias=0, activation='relu', input_shape=(784,)))
#model.add(Dropout(0.2))
model2.add(Dense(256, use_bias=0, activation='relu'))
model2.add(Dense(256, use_bias=0, activation='relu'))
#model.add(Dropout(0.2))
model2.add(Dense(num_classes, use_bias=0, activation='softmax'))

model2.load_weights('my_model_weights.h5')


In [ ]:
# functions for doing inference in a for loop


@jit
def quantize_arr4(arr, max_val):
    """Quantization based on linear rescaling over min/max range.
    """
    quantized = np.round(4 * (arr/max_val))
    quantized /= 4
    quantized *= max_val
    return quantized
@jit
def feedforward(inputs,weights,absmax,activate_relu=1,leaf=64,spread_branch=1/256):#inputs.shape[0]=784
    inputs = inputs * np.random.uniform(1-tl_variation(),1+tl_variation(),inputs.shape)
    
    copy_num = weights[0].shape[1]
    spliter_count = math.log2(leaf) + 3

    random_input = np.repeat(inputs,copy_num,0).reshape(inputs.shape[0],copy_num) * spliter_forest(inputs.shape[0],copy_num)

    
    random_input *= coupling_loss()    
       
    weights_random = weights[0] * np.random.uniform(0.95,1.05,weights[0].shape)
    
    result = random_input * weights_random * propagation_loss() * splitter_loss()
   
    random_input[np.abs(random_input)<1e-3]=0
    
    result = result * 1/2 * splitter_loss()
    
    result[np.abs(result)<2e-7]=0
    
    result = np.sum (result,0) 
    
    output = amplifier(result , 2 * absmax * leaf /(1 - spread_branch) / coupling_loss()/\
                       splitter_loss() ** spliter_count/propagation_loss() ** (spliter_count+1)) * propagation_loss() 
    
    if activate_relu==1:
        output=relu(output)
        output[output>1]=1
        
    return output

@jit
def relu(output):
  nextinput = np.zeros((output.shape[0]))
  for i in range(nextinput.shape[0]):
    nextinput[i] = max(0,output[i])
  return nextinput

@jit
def normalize(array,quanti=8):#normalize and quantize the weight, denormalize in feedfoward
    w=np.asarray(array)
    absmax=np.amax(np.abs(w))
    w /= absmax
    quantScale=1/quanti/2
    for i in range(-quanti,quanti):
        w[np.abs(w-i/quanti)<=quantScale] = i/quanti
    w[np.abs(w-quantScale/2)<quantScale/2]=1/quanti
    w[np.abs(w-(1-quantScale/2))<quantScale/2]=(quanti-1)/quanti
    return w
                       

@jit
def network(x_test, y_test):#x_test is 1*784, one 28*28 image
    weights0=np.asarray(model2.layers[0].get_weights())
    absmax=np.amax(np.abs(weights0))
    o1 = feedforward(x_test,normalize(weights0),absmax)
    weights1=np.asarray(model2.layers[1].get_weights())
    absmax=np.amax(np.abs(weights1))
    o1 = feedforward(o1,normalize(weights1),absmax)
    weights2=np.asarray(model2.layers[2].get_weights())
    absmax=np.amax(np.abs(weights2))
    o1 = feedforward(o1,normalize(weights2),absmax)
    weights3=np.asarray(model2.layers[3].get_weights())
    absmax=np.amax(np.abs(weights3))
    o1 = feedforward(o1,normalize(weights3),absmax,0)
    pred_out = np.zeros((1,o1.shape[0]))
    pred_out[0] = o1
    correct_out = np.zeros((1,y_test.shape[0]))
    correct_out[0] = y_test
    return (np.argmax(pred_out) == np.argmax(correct_out))
@jit
def spliter_tree(root_value=1,leaf=64,variation=0.05):
    height=math.ceil(math.log2(leaf))
    arr=[]
    for i in range(0,height+1):
        arr.append([])
        node=2**i
        if i==0:
            arr[i].append(root_value)
        else:
            for j in range(0,int(node/2)):
                    v=np.random.uniform(1-variation,1+variation)#inject splitter loss here
                    arr[i].append(v*(1/2)*arr[i-1][math.floor(j)]*propagation_loss()*splitter_loss())
                    arr[i].append((2-v)*(1/2)*arr[i-1][math.floor(j)]*propagation_loss()*splitter_loss())
    return np.asarray(arr[-1])
@jit
def spliter_grove(total=256,leaf=64,spread_branch=1/256,variation=0.05):
    tree_num=math.ceil(total/leaf)
    arr=np.zeros((tree_num,leaf))
    v=np.random.uniform(1-variation,1+variation,tree_num)
    right=np.zeros(tree_num)
    left=np.zeros(tree_num)
    right[0]=v[0]*spread_branch
    left[0]=1-right[0]
    arr[0]=spliter_tree(left[0]*propagation_loss()*splitter_loss(),leaf,variation)
    for i in range(1,tree_num):
        right[i]=amplifier(right[i-1]*splitter_loss()*propagation_loss(),1/(spread_branch*splitter_loss()*propagation_loss()))\
        *v[i]*spread_branch
        left[i]=1-right[i]
        arr[i]=spliter_tree(left[i]*propagation_loss()*splitter_loss(),leaf,variation)
    return arr.flatten()[:total] 
@jit
def spliter_forest(input_num=784,grove_len=256,leaf=64):
    arr=np.zeros((input_num,grove_len))
    for i in range(0,input_num):
        arr[i]=spliter_grove(grove_len)
    return arr 
def amplifier(x, multiple=64, amp_vari=0.05):
    x = x / 6.25
    delta_in= x * np.random.uniform(-amp_vari,amp_vari)
    multiple *= 6.25
    x_out = x * multiple + multiple * delta_in*10**(1/2) # x * multiple or x_in * multiple ?
    return x_out
def propagation_loss(length=15*10e-4,loss_per_length=0.3):
    return 1/10**(length*loss_per_length/10)
def splitter_loss(loss=0.2):
    return 1/10**(loss/10)
def coupling_loss(layers=2):
    return 1/10**(layers*0.5/10)
def tl_variation(vari=0.05):
    return vari


In [ ]:
@jit
def main(logname='newest.txt'):
    log=open(logname,'w')
    log.write(time.strftime("%Y-%m-%d %H:%M:%S\n", time.localtime()))
    log.close()
    ave=np.ones(100)
    (x_train1, y_train1), (x_test1, y_test1) = mnist.load_data()
    x_test1 = x_test1.reshape(10000, 784)
    x_test1 = x_test1.astype('float32')
    x_test1 /= np.amax(x_test1)
    x_test1 = quantize_arr4(x_test1,np.amax(x_test1))
    y_test1 = keras.utils.to_categorical(y_test1, num_classes)
    total_test = x_test1.shape[0]
    for j in range(0,100):
        correct_test = 0        
        for i in range(x_test1.shape[0]):
          if (network(x_test1[i],y_test1[i])):
             correct_test += 1
          if (i+1)%10==0: # to test the function
             print ('inference %d current accuracy at %i:'%(j+1,i+1),correct_test/(i+1))
        print ('The accuracy of inference %d: %f'%(j+1,correct_test/float(total_test)))
        ave[j]=correct_test/float(total_test)
        log=open(logname,'a')
        log.write('The accuracy of inference %d: %f;\n'%(j+1,correct_test/float(total_test)))
        log.close()
    log=open(logname,'a')
    log.write('The mean of accuracy is %f.\n'%np.average(ave))
    log.write('The variance of accuracy is %f.\n'%np.var(ave))
    log.close()
main('12_4.txt')